In [2]:
!pip install --upgrade gdown
import gdown

# https://drive.google.com/file/d/1crMjnsoE0DNNqBzxp8Zt7PGXW3GL8tFO/view?usp=drive_link
file_id = "1crMjnsoE0DNNqBzxp8Zt7PGXW3GL8tFO"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="[F1]TekstoviCSV_JSON.zip", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1crMjnsoE0DNNqBzxp8Zt7PGXW3GL8tFO
From (redirected): https://drive.google.com/uc?id=1crMjnsoE0DNNqBzxp8Zt7PGXW3GL8tFO&confirm=t&uuid=777f9f2c-1432-4808-bb2a-efeeb459301f
To: /content/[F1]TekstoviCSV_JSON.zip
100%|██████████| 698M/698M [00:13<00:00, 53.0MB/s]


'[F1]TekstoviCSV_JSON.zip'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import zipfile

with zipfile.ZipFile("[F1]TekstoviCSV_JSON.zip", 'r') as zip_ref:
    zip_ref.extractall("[F1]TekstoviCSV_JSON")


In [ ]:
!python -c "import nltk; nltk.download('punkt')"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install sumy transformers datasets torch pandas

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
import csv
from pathlib import Path
from transformers import pipeline

# --- Setup ---
input_path = Path("[F1]TekstoviCSV_JSON/buka.ba")
output_path = Path("[F2-4]")
output_path.mkdir(parents=True, exist_ok=True)

abstractive_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# --- Metrics ---
def compute_metrics(original_text: str, summary_text: str) -> dict:
    compression = len(summary_text) / len(original_text) if len(original_text) > 0 else 0
    original_words = set(original_text.lower().split())
    summary_words = set(summary_text.lower().split())
    overlap = original_words.intersection(summary_words)
    coverage = len(overlap) / len(summary_words) if len(summary_words) > 0 else 0
    density = sum([1 for word in summary_words if word in original_words]) / len(summary_words) if len(summary_words) > 0 else 0
    return {
        "Compression": round(compression * 100, 2),
        "Coverage": round(coverage, 2),
        "Density": round(density * 100, 2),
        "Compression_bin": "low" if compression > 0.4 else "high",
        "Coverage_bin": "high" if coverage > 0.5 else "low",
        "Density_bin": "extractive" if density > 0.6 else "abstractive",
    }

# --- Sumarizacija i spremanje ---
def summarize_file(txt_path: Path):
    with open(txt_path, "r", encoding="utf-8") as f:
        text = f.read()

    # Ekstraktivna sumarizacija: prve 3 rečenice kao najjednostavnija zamjena
    extractive_summary = ". ".join(text.split(".")[:3]).strip() + "."

    # Abstraktivna sumarizacija pomoću transformer modela
    abstractive_result = abstractive_summarizer(text[:1024], max_length=150, min_length=30, do_sample=False)
    abstractive_summary = abstractive_result[0]['summary_text']

    extractive_metrics = compute_metrics(text, extractive_summary)
    abstractive_metrics = compute_metrics(text, abstractive_summary)

    return {
        "filename": txt_path.name,
        "filepath": str(txt_path),
        "text": text,
        "extractive_summary": extractive_summary,
        "abstractive_summary": abstractive_summary,
        "extractive_metrics": extractive_metrics,
        "abstractive_metrics": abstractive_metrics,
    }

# --- Glavna petlja ---
results = []

for txt_file in input_path.rglob("*.txt"):
    try:
        summary_data = summarize_file(txt_file)
        results.append(summary_data)
    except Exception as e:
        print(f"Greska u fajlu {txt_file}: {e}")

# --- Spremanje rezultata ---
json_output = output_path / "summary_results.json"
csv_output = output_path / "summary_results.csv"

with open(json_output, "w", encoding="utf-8") as jf:
    json.dump(results, jf, ensure_ascii=False, indent=2)

with open(csv_output, "w", newline="", encoding="utf-8") as cf:
    fieldnames = [
        "filename", "filepath", "extractive_summary", "abstractive_summary",
        "Compression", "Coverage", "Density",
        "Compression_bin", "Coverage_bin", "Density_bin"
    ]
    writer = csv.DictWriter(cf, fieldnames=fieldnames)
    writer.writeheader()
    for res in results:
        for kind in ["extractive", "abstractive"]:
            metrics = res[f"{kind}_metrics"]
            writer.writerow({
                "filename": res["filename"] + f" ({kind})",
                "filepath": res["filepath"],
                "extractive_summary": res["extractive_summary"] if kind == "extractive" else "",
                "abstractive_summary": res["abstractive_summary"] if kind == "abstractive" else "",
                **metrics
            })

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [9]:
# -*- coding: utf-8 -*-
"""
# Jednostavna sumarizacija teksta bez NLTK
"""

# Instaliranje potrebnih biblioteka
!pip install scikit-learn pandas

import os
import json
import csv
import re
import pandas as pd
import numpy as np
from collections import Counter
from google.colab import drive
from sklearn.feature_extraction.text import TfidfVectorizer

# Povezivanje Google Drive-a
drive.mount('/content/drive')

# Bosanske stop riječi - riječi koje nemaju veliki značaj za sadržaj
STOP_RIJECI = [
    'i', 'u', 'je', 'da', 'se', 'na', 'su', 'za', 'od', 'što', 'kako', 'ali', 'a', 'ili',
    'to', 'nije', 'sam', 'mi', 'ti', 'on', 'ona', 'ono', 'oni', 'one', 'smo', 'ste', 'sa',
    'jer', 'po', 'kad', 'kao', 'do', 'iz', 'bi', 'te', 'tako', 'pri', 'nakon', 'već', 'o',
    'kroz', 'još', 'samo', 'sve', 'ako', 'mogu', 'može', 'biti', 'bio', 'bila', 'bilo',
    'bili', 'bile', 'će', 'ću', 'ćemo', 'ćete', 'neće', 'ne', 'niti', 'pa', 'dok',
    'prije', 'ovaj', 'ova', 'ovo', 'ovi', 'ove', 'taj', 'ta', 'to', 'ti', 'te', 'svoj',
    'svoja', 'svoje', 'svoji', 'svoje', 'moj', 'moja', 'moje', 'moji', 'moje'
]

# VAŽNO: Prilagodite ove putanje prema stvarnoj strukturi vašeg Google Drive-a
# Provjerite tačan put i strukturu foldera
BASE_DIR = '/content/drive/MyDrive'  # Ovo je standardna putanja, ali možda je drugačije kod vas

# Funkcija za pronalaženje ispravne putanje
def find_correct_path():
    print("Tražim ispravnu putanju za [F1]TekstoviCSV_JSON...")

    # Provjerite nekoliko mogućih lokacija
    potential_paths = [
        '/content/drive/MyDrive/[F1]TekstoviCSV_JSON',
        '/content/drive/MyDrive/F1TekstoviCSV_JSON',
        '/content/drive/My Drive/[F1]TekstoviCSV_JSON',
        '/content/drive/My Drive/F1TekstoviCSV_JSON',
        '/content/drive/Shared drives/[F1]TekstoviCSV_JSON',
        '/content/drive/Shared drives/F1TekstoviCSV_JSON'
    ]

    # Provjera svih putanja
    for path in potential_paths:
        print(f"Provjeravam: {path}")
        if os.path.exists(path):
            print(f"Pronađena putanja: {path}")
            return path

    # Ako ne možemo naći tačnu lokaciju, ispišemo sadržaj root foldera
    print("\nNe mogu pronaći folder. Evo sadržaja vašeg drive-a:")
    root_contents = os.listdir('/content/drive')
    print(f"Root sadržaj: {root_contents}")

    if 'MyDrive' in root_contents or 'My Drive' in root_contents:
        mydrive_path = '/content/drive/MyDrive' if 'MyDrive' in root_contents else '/content/drive/My Drive'
        print(f"\nSadržaj vašeg MyDrive-a:")
        try:
            mydrive_contents = os.listdir(mydrive_path)
            for item in mydrive_contents:
                print(f"  - {item}")
        except Exception as e:
            print(f"Ne mogu pristupiti: {e}")

    # Tražimo unos od korisnika
    user_path = input("\nUnesite tačnu putanju do vašeg [F1]TekstoviCSV_JSON foldera (kopirajte putanju): ")
    if os.path.exists(user_path):
        return user_path
    else:
        print(f"Putanja {user_path} ne postoji. Koristiću standardnu putanju.")
        return '/content/drive/MyDrive/[F1]TekstoviCSV_JSON'

# Funkcija za podjelu teksta na rečenice - jednostavna zamjena za NLTK sent_tokenize
def podijeli_na_recenice(text):
    # Razdvajamo tekst na rečenice - tražimo znakove ., ! i ?
    text = text.replace('!', '.').replace('?', '.')
    # Rješavamo problem s tačkama u skraćenicama (npr., dr., itd., npr.)
    text = re.sub(r'(?<=[a-zčćžšđ])\.(?=[A-ZČĆŽŠĐ])', '. ', text)
    # Podjela na rečenice
    recenice = [s.strip() for s in text.split('.') if s.strip()]
    return recenice

# Funkcija za ekstraktivnu sumarizaciju (izvlačenje važnih rečenica)
def ekstraktivna_sumarizacija(text, broj_recenica=3):
    if not text or text.strip() == "":
        return "Nema dostupnog teksta za sumarizaciju."

    # Podjela na rečenice
    recenice = podijeli_na_recenice(text)

    # Ako imamo manje od traženog broja rečenica, vratimo sve
    if len(recenice) <= broj_recenica:
        return ". ".join(recenice) + "."

    # Računanje važnosti riječi (bez stop riječi)
    sve_rijeci = []
    for recenica in recenice:
        rijeci = [rijec.lower() for rijec in re.findall(r'\b\w+\b', recenica)
                if rijec.lower() not in STOP_RIJECI and len(rijec) > 2]
        sve_rijeci.extend(rijeci)

    # Frekvencija riječi
    frekvencija_rijeci = Counter(sve_rijeci)

    # Računanje skora za svaku rečenicu
    skorovi_recenica = []
    for i, recenica in enumerate(recenice):
        rijeci_u_recenici = [rijec.lower() for rijec in re.findall(r'\b\w+\b', recenica)]
        skor = sum(frekvencija_rijeci[rijec] for rijec in rijeci_u_recenici
                  if rijec in frekvencija_rijeci)

        # Dajemo veći skor prvim rečenicama jer često sadrže važne informacije
        if i == 0:
            skor *= 1.5
        elif i == 1:
            skor *= 1.2

        skorovi_recenica.append((skor, i, recenica))

    # Sortiramo po skoru
    skorovi_recenica.sort(reverse=True)

    # Uzimamo top N rečenica prema originalnom redoslijedu
    odabrani_indeksi = sorted([item[1] for item in skorovi_recenica[:broj_recenica]])
    odabrane_recenice = [recenice[i] for i in odabrani_indeksi]

    # Spajamo u jedan paragraf
    return ". ".join(odabrane_recenice) + "."

# Funkcija za abstraktivnu sumarizaciju (generisanje novog teksta)
def abstraktivna_sumarizacija(text):
    if not text or text.strip() == "":
        return "Nema dostupnog teksta za sumarizaciju."

    # Podjela na rečenice
    recenice = podijeli_na_recenice(text)

    if not recenice:
        return "Nije moguće generisati sažetak."

    # Izdvajanje ključnih riječi iz teksta
    sve_rijeci = []
    for recenica in recenice:
        rijeci = [rijec.lower() for rijec in re.findall(r'\b\w+\b', recenica)
                if rijec.lower() not in STOP_RIJECI and len(rijec) > 2]
        sve_rijeci.extend(rijeci)

    # Dobijanje najčešćih riječi
    frekvencija_rijeci = Counter(sve_rijeci)
    kljucne_rijeci = [rijec for rijec, _ in frekvencija_rijeci.most_common(10)]

    # Pronalazimo rečenice koje sadrže najviše ključnih riječi
    skorovi_recenica = []
    for recenica in recenice:
        rijeci_u_recenici = [rijec.lower() for rijec in re.findall(r'\b\w+\b', recenica)]
        skor = sum(1 for rijec in kljucne_rijeci if rijec in rijeci_u_recenici)
        skorovi_recenica.append((skor, recenica))

    # Sortiramo po skoru
    skorovi_recenica.sort(reverse=True)

    # Odabiremo top 2-3 rečenice za sažetak
    odabrane_recenice = []
    for _, recenica in skorovi_recenica[:3]:
        # Za abstraktivnu sumarizaciju, radimo jednostavnu transformaciju rečenica
        rijeci = recenica.split()
        if len(rijeci) > 10:
            # Skraćujemo duge rečenice
            nova_recenica = " ".join(rijeci[:len(rijeci)//2]) + "..."
            odabrane_recenice.append(nova_recenica)
        else:
            odabrane_recenice.append(recenica)

    # Spajamo u jedan paragraf
    return ". ".join(odabrane_recenice) + "."

# Funkcija za procesiranje JSON fajla
def procesiraj_json(ulazni_fajl, izlazni_fajl):
    try:
        # Učitavanje JSON
        with open(ulazni_fajl, 'r', encoding='utf-8') as f:
            podaci = json.load(f)

        # Provjera da li je podaci lista (array) ili pojedinačni objekt
        if isinstance(podaci, list):
            # Ako je lista, procesiramo svaki objekt u listi
            for i, item in enumerate(podaci):
                if 'Text' in item and item['Text']:
                    tekst = item['Text']

                    # Generisanje oba tipa sumarizacije
                    eks_sazetak = ekstraktivna_sumarizacija(tekst, 3)
                    abs_sazetak = abstraktivna_sumarizacija(tekst)

                    # Dodavanje novih polja
                    item['ExtractiveSum'] = eks_sazetak
                    item['AbstractiveSum'] = abs_sazetak
                else:
                    item['ExtractiveSum'] = ""
                    item['AbstractiveSum'] = ""
        else:
            # Postojeća logika za pojedinačne objekte
            if 'Text' in podaci and podaci['Text']:
                tekst = podaci['Text']

                # Generisanje oba tipa sumarizacije
                eks_sazetak = ekstraktivna_sumarizacija(tekst, 3)
                abs_sazetak = abstraktivna_sumarizacija(tekst)

                # Dodavanje novih polja
                podaci['ExtractiveSum'] = eks_sazetak
                podaci['AbstractiveSum'] = abs_sazetak
            else:
                podaci['ExtractiveSum'] = ""
                podaci['AbstractiveSum'] = ""

        # Zapis novog JSON-a
        os.makedirs(os.path.dirname(izlazni_fajl), exist_ok=True)
        with open(izlazni_fajl, 'w', encoding='utf-8') as f:
            json.dump(podaci, f, ensure_ascii=False, indent=4)

        return True
    except Exception as e:
        print(f"Greška pri obradi JSON fajla {ulazni_fajl}: {e}")
        return False

# Funkcija za procesiranje CSV fajla
def procesiraj_csv(ulazni_fajl, izlazni_fajl):
    try:
        # Čitanje CSV-a u pandas DataFrame
        df = pd.read_csv(ulazni_fajl, encoding='utf-8')

        # Provjera da li postoji kolona Text
        if 'Text' in df.columns:
            # Generisanje sažetaka za svaki red
            df['ExtractiveSum'] = df['Text'].apply(lambda x: ekstraktivna_sumarizacija(str(x), 3) if pd.notna(x) else "")
            df['AbstractiveSum'] = df['Text'].apply(lambda x: abstraktivna_sumarizacija(str(x)) if pd.notna(x) else "")
        else:
            df['ExtractiveSum'] = ""
            df['AbstractiveSum'] = ""

        # Zapis novog CSV-a
        os.makedirs(os.path.dirname(izlazni_fajl), exist_ok=True)
        df.to_csv(izlazni_fajl, index=False, encoding='utf-8')

        return True
    except Exception as e:
        print(f"Greška pri obradi CSV fajla {ulazni_fajl}: {e}")

        try:
            # Alternativni pristup ako pandas ne radi
            redovi = []
            zaglavlja = []

            # Čitanje CSV-a ručno
            with open(ulazni_fajl, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                zaglavlja = next(reader)  # Prva linija su zaglavlja

                # Dodavanje novih polja za sumarizacije
                if 'ExtractiveSum' not in zaglavlja:
                    zaglavlja.append('ExtractiveSum')
                if 'AbstractiveSum' not in zaglavlja:
                    zaglavlja.append('AbstractiveSum')

                # Indeks Text kolone
                try:
                    tekst_indeks = zaglavlja.index('Text')
                except ValueError:
                    tekst_indeks = -1

                # Obrada redova
                for red in reader:
                    # Ako nemamo dovoljno kolona, proširimo red
                    while len(red) < len(zaglavlja) - 2:  # -2 za nove kolone
                        red.append("")

                    # Generisanje sažetaka ako imamo tekst
                    if tekst_indeks >= 0 and tekst_indeks < len(red) and red[tekst_indeks]:
                        tekst = red[tekst_indeks]
                        eks_sazetak = ekstraktivna_sumarizacija(tekst, 3)
                        abs_sazetak = abstraktivna_sumarizacija(tekst)
                    else:
                        eks_sazetak = ""
                        abs_sazetak = ""

                    # Dodavanje sažetaka u red
                    red.append(eks_sazetak)
                    red.append(abs_sazetak)
                    redovi.append(red)

            # Zapis novog CSV-a
            os.makedirs(os.path.dirname(izlazni_fajl), exist_ok=True)
            with open(izlazni_fajl, 'w', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(zaglavlja)
                writer.writerows(redovi)

            return True
        except Exception as e2:
            print(f"Greška pri alternativnom pristupu za CSV: {e2}")
            return False

# Glavna funkcija za obradu svih fajlova
def obradi_sve_fajlove(ulazni_folder, izlazni_folder):
    # Pronađimo podfolder buka.ba
    buka_folder = os.path.join(ulazni_folder, "buka.ba")

    if not os.path.exists(buka_folder):
        print(f"GREŠKA: Folder {buka_folder} ne postoji!")

        # Pokušajmo pronaći tačnu strukturu
        print("Tražim buka.ba folder...")

        # Ispisujemo sadržaj ulaznog foldera za dijagnostiku
        print(f"Sadržaj {ulazni_folder}:")
        try:
            for item in os.listdir(ulazni_folder):
                print(f"  - {item}")
                item_path = os.path.join(ulazni_folder, item)
                if os.path.isdir(item_path):
                    print(f"    Podfolderi u {item}:")
                    try:
                        for subitem in os.listdir(item_path):
                            print(f"      - {subitem}")
                    except Exception as e:
                        print(f"      Greška pri čitanju: {e}")
        except Exception as e:
            print(f"Greška pri čitanju direktorija: {e}")

        # Tražimo alternativne putanje
        alternative = [
            os.path.join(ulazni_folder, "buka_ba"),
            ulazni_folder  # Možda je glavni folder već buka.ba?
        ]

        for alt_path in alternative:
            if os.path.exists(alt_path):
                print(f"Pronađena alternativna putanja: {alt_path}")
                buka_folder = alt_path
                break

    # Kreirajmo izlazni folder
    izlazni_buka_folder = os.path.join(izlazni_folder, "buka.ba")
    os.makedirs(izlazni_buka_folder, exist_ok=True)

    processed_files = 0
    errors = 0

    # Prolazak kroz podfoldere
    print(f"Obrađujem fajlove iz {buka_folder}...")

    try:
        for item in os.listdir(buka_folder):
            item_path = os.path.join(buka_folder, item)

            if os.path.isdir(item_path):
                print(f"Obrada direktorija: {item}")

                # Kreiranje odgovarajućeg izlaznog podfoldera
                izlazni_podfolder = os.path.join(izlazni_buka_folder, item)
                os.makedirs(izlazni_podfolder, exist_ok=True)

                # Obrada fajlova u podfolderu
                for filename in os.listdir(item_path):
                    ulazni_fajl = os.path.join(item_path, filename)
                    izlazni_fajl = os.path.join(izlazni_podfolder, filename)

                    if filename.endswith('.json'):
                        print(f"  Obrada JSON: {filename}")
                        if procesiraj_json(ulazni_fajl, izlazni_fajl):
                            processed_files += 1
                        else:
                            errors += 1

                    elif filename.endswith('.csv'):
                        print(f"  Obrada CSV: {filename}")
                        if procesiraj_csv(ulazni_fajl, izlazni_fajl):
                            processed_files += 1
                        else:
                            errors += 1

            elif os.path.isfile(item_path):
                # Ako su fajlovi direktno u buka.ba folderu
                filename = item
                ulazni_fajl = item_path
                izlazni_fajl = os.path.join(izlazni_buka_folder, filename)

                if filename.endswith('.json'):
                    print(f"Obrada JSON: {filename}")
                    if procesiraj_json(ulazni_fajl, izlazni_fajl):
                        processed_files += 1
                    else:
                        errors += 1

                elif filename.endswith('.csv'):
                    print(f"Obrada CSV: {filename}")
                    if procesiraj_csv(ulazni_fajl, izlazni_fajl):
                        processed_files += 1
                    else:
                        errors += 1

    except Exception as e:
        print(f"Globalna greška pri obradi: {e}")

    return processed_files, errors

# Glavni program
if __name__ == "__main__":
    print("\n" + "=" * 50)
    print("SUMARIZACIJA BOSANSKIH TEKSTOVA (BEZ NLTK)")
    print("=" * 50)

    # Pronalaženje ispravne putanje do ulaznih fajlova
    INPUT_FOLDER = find_correct_path()
    OUTPUT_FOLDER = os.path.join(BASE_DIR, "[F2]SumarizovaniTekstovi")

    print(f"Ulazni folder: {INPUT_FOLDER}")
    print(f"Izlazni folder: {OUTPUT_FOLDER}")
    print("=" * 50)

    # Kreiranje izlaznog foldera
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f"Kreiran izlazni folder: {OUTPUT_FOLDER}")

    # Obrada svih fajlova
    processed, errors = obradi_sve_fajlove(INPUT_FOLDER, OUTPUT_FOLDER)

    print("\n" + "=" * 50)
    print(f"Obrada završena!")
    print(f"Uspješno obrađeno: {processed} fajlova")
    print(f"Greške: {errors}")
    print(f"Rezultati su sačuvani u folderu: {OUTPUT_FOLDER}")
    print("=" * 50)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

SUMARIZACIJA BOSANSKIH TEKSTOVA (BEZ NLTK)
Tražim ispravnu putanju za [F1]TekstoviCSV_JSON...
Provjeravam: /content/drive/MyDrive/[F1]TekstoviCSV_JSON
Provjeravam: /content/drive/MyDrive/F1TekstoviCSV_JSON
Provjeravam: /content/drive/My Drive/[F1]TekstoviCSV_JSON
Provjeravam: /content/drive/My Drive/F1TekstoviCSV_JSON
Provjeravam: /content/drive/Shared drives/[F1]TekstoviCSV_JSON
Provjeravam: /content/drive/Shared drives/F1TekstoviCSV_JSON

Ne mogu pronaći folder. Evo sadržaja vašeg drive-a:
Root sadržaj: ['MyDrive', '.shortcut-targets-by-id', 'Shareddrives', '.file-revisions-by-id', '.Trash-0']

Sadržaj vašeg MyDrive-a:
  - VI_2025
  - [F2]SumarizovaniTekstovi

Unesite tačnu putanju do vašeg [F1]TekstoviCSV_JSON foldera (kopirajte putanju): /content/[F1]TekstoviCSV_JSON
Ulazni folder: /content/[F1]TekstoviCSV_JSON
Izlazni folder: /content/drive/MyDrive/[F2]

In [10]:
# Ispod je kod za zipovanje te instaliranje dobivenog foldera lokalno
import shutil
shutil.make_archive("/content/sumarizovani_tekstovi", 'zip', "/content/drive/MyDrive/[F2]SumarizovaniTekstovi")


'/content/sumarizovani_tekstovi.zip'

In [11]:
from google.colab import files
files.download("/content/sumarizovani_tekstovi.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>